분류문제 (binary classification)
hypothesis : x가 1일 확률

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.manual_seed(1)

In [2]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]

x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [3]:
W = torch.zeros((2,1), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

In [4]:
hypothesis = 1/(1+torch.exp(-(x_train.matmul(W)+b)))

In [5]:
print(hypothesis, hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>) torch.Size([6, 1])


Cross Entropy Error 사용!

In [7]:
cost = F.binary_cross_entropy(hypothesis, y_train)
print(cost)

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward>)


In [16]:
#train

W = torch.zeros((2,1), requires_grad = True)
b = torch.zeros(1, requires_grad = True)
optimizer = optim.SGD([W,b],lr = 1)
epochs = 1000
for epoch in range(epochs+1):
    hypothesis = torch.sigmoid(x_train.matmul(W)+b)
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch%50 ==0:
        print('epoch {:4d}/{} Cost: {:.6f}'.format(epoch, epochs, cost.item()))

epoch    0/1000 Cost: 0.693147
epoch   50/1000 Cost: 0.319570
epoch  100/1000 Cost: 0.134722
epoch  150/1000 Cost: 0.100731
epoch  200/1000 Cost: 0.080643
epoch  250/1000 Cost: 0.067357
epoch  300/1000 Cost: 0.057900
epoch  350/1000 Cost: 0.050814
epoch  400/1000 Cost: 0.045300
epoch  450/1000 Cost: 0.040882
epoch  500/1000 Cost: 0.037261
epoch  550/1000 Cost: 0.034237
epoch  600/1000 Cost: 0.031672
epoch  650/1000 Cost: 0.029469
epoch  700/1000 Cost: 0.027556
epoch  750/1000 Cost: 0.025878
epoch  800/1000 Cost: 0.024394
epoch  850/1000 Cost: 0.023073
epoch  900/1000 Cost: 0.021888
epoch  950/1000 Cost: 0.020820
epoch 1000/1000 Cost: 0.019852


In [17]:
# evaluation
# 여기서는 test셋이 따로 없어서 train으로 확인
result = torch.sigmoid(x_train.matmul(W)+b)
print(result)

tensor([[2.7648e-04],
        [3.1608e-02],
        [3.8977e-02],
        [9.5622e-01],
        [9.9823e-01],
        [9.9969e-01]], grad_fn=<SigmoidBackward>)


# Higher Implementation with Class (실전 구현)

In [23]:
class BinaryClassifier(nn.Module):   #nn.Module이라는 추상클래스를 상속받음
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2,1) #W와 b가 둘다 들어있는 Linear layer
        self.sigmoid = nn.Sigmoid()
    
    def forward(self,x):
        return self.sigmoid(self.linear(x))
    
model = BinaryClassifier()

In [24]:
optimizer = optim.SGD(model.parameters(), lr = 1)
epochs = 100
for epoch in range(epochs+1):
    hypothesis = model(x_train)
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch%10 ==0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_predic = prediction.float()==y_train
        accuracy = correct_predic.sum().item()/len(correct_predic)
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}'.format(epoch, epochs, cost.item(),accuracy*100))

Epoch    0/100 Cost: 0.733120 Accuracy 50.00
Epoch   10/100 Cost: 0.471201 Accuracy 83.33
Epoch   20/100 Cost: 0.739920 Accuracy 66.67
Epoch   30/100 Cost: 0.542286 Accuracy 83.33
Epoch   40/100 Cost: 0.451817 Accuracy 83.33
Epoch   50/100 Cost: 0.367631 Accuracy 83.33
Epoch   60/100 Cost: 0.288507 Accuracy 83.33
Epoch   70/100 Cost: 0.220185 Accuracy 83.33
Epoch   80/100 Cost: 0.173718 Accuracy 100.00
Epoch   90/100 Cost: 0.151886 Accuracy 100.00
Epoch  100/100 Cost: 0.140431 Accuracy 100.00
